<a href="https://colab.research.google.com/github/66Fenix66/moon/blob/main/Thoracic_surgery_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import VarianceThreshold
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.model_selection import learning_curve, GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn import tree

In [ ]:
from google.colab import files
files.upload()


{}

In [ ]:
tds = pd.read_csv(r'ThoraricSurgery.csv')

In [ ]:
tds.head()

,id,DGN,PRE4,PRE5,PRE6,PRE7,PRE8,PRE9,PRE10,PRE11,PRE14,PRE17,PRE19,PRE25,PRE30,PRE32,AGE,Risk1Yr
0,1,DGN2,2.88,2.16,PRZ1,F,F,F,T,T,OC14,F,F,F,T,F,60,F
1,2,DGN3,3.40,1.88,PRZ0,F,F,F,F,F,OC12,F,F,F,T,F,51,F
2,3,DGN3,2.76,2.08,PRZ1,F,F,F,T,F,OC11,F,F,F,T,F,59,F
3,4,DGN3,3.68,3.04,PRZ0,F,F,F,F,F,OC11,F,F,F,F,F,54,F
4,5,DGN3,2.44,0.96,PRZ2,F,T,F,T,T,OC11,F,F,F,T,F,73,T


ап

In [ ]:
tds = tds.drop('id', axis=1)
tds.isnull().sum()

DGN        0
PRE4       0
PRE5       0
PRE6       0
PRE7       0
PRE8       0
PRE9       0
PRE10      0
PRE11      0
PRE14      0
PRE17      0
PRE19      0
PRE25      0
PRE30      0
PRE32      0
AGE        0
Risk1Yr    0
dtype: int64

In [ ]:
tds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DGN      470 non-null    object 
 1   PRE4     470 non-null    float64
 2   PRE5     470 non-null    float64
 3   PRE6     470 non-null    object 
 4   PRE7     470 non-null    object 
 5   PRE8     470 non-null    object 
 6   PRE9     470 non-null    object 
 7   PRE10    470 non-null    object 
 8   PRE11    470 non-null    object 
 9   PRE14    470 non-null    object 
 10  PRE17    470 non-null    object 
 11  PRE19    470 non-null    object 
 12  PRE25    470 non-null    object 
 13  PRE30    470 non-null    object 
 14  PRE32    470 non-null    object 
 15  AGE      470 non-null    int64  
 16  Risk1Yr  470 non-null    object 
dtypes: float64(2), int64(1), object(14)
memory usage: 62.5+ KB


In [ ]:
tds[['PRE7', 'PRE8', 'PRE9', 'PRE10', 'PRE11', 'PRE17', 'PRE19', 'PRE25', 'PRE30', 'PRE32', 'Risk1Yr']] = \
(tds[['PRE7', 'PRE8', 'PRE9', 'PRE10', 'PRE11', 'PRE17', 'PRE19', 'PRE25', 'PRE30', 'PRE32', 'Risk1Yr']] == 'T').astype(int)
tds['DGN']   = tds['DGN'].str[-1:].astype(int)
tds['PRE6']  = tds['PRE6'].str[-1:].astype(int)
tds['PRE14'] = tds['PRE14'].str[-1:].astype(int)

In [ ]:
tds.head()

,DGN,PRE4,PRE5,PRE6,PRE7,PRE8,PRE9,PRE10,PRE11,PRE14,PRE17,PRE19,PRE25,PRE30,PRE32,AGE,Risk1Yr
0,2,2.88,2.16,1,0,0,0,1,1,4,0,0,0,1,0,60,0
1,3,3.40,1.88,0,0,0,0,0,0,2,0,0,0,1,0,51,0
2,3,2.76,2.08,1,0,0,0,1,0,1,0,0,0,1,0,59,0
3,3,3.68,3.04,0,0,0,0,0,0,1,0,0,0,0,0,54,0
4,3,2.44,0.96,2,0,1,0,1,1,1,0,0,0,1,0,73,1


In [ ]:
columns = ['Диагноз','Ёмкость_легких','Объем_выдоха','Шкала_Зуброда','Боль','Кровохарканье','Одышка',
       'Кашель','Слабость','Размер_опухоли','Диабет','ИМ_6_мес','Заболевание_артерий','Курение','Астма','Возраст','Выживание_год']
tds.columns = columns

In [ ]:
tds.head(3)

,Диагноз,Ёмкость_легких,Объем_выдоха,Шкала_Зуброда,Боль,Кровохарканье,Одышка,Кашель,Слабость,Размер_опухоли,Диабет,ИМ_6_мес,Заболевание_артерий,Курение,Астма,Возраст,Выживание_год
0,2,2.88,2.16,1,0,0,0,1,1,4,0,0,0,1,0,60,0
1,3,3.40,1.88,0,0,0,0,0,0,2,0,0,0,1,0,51,0
2,3,2.76,2.08,1,0,0,0,1,0,1,0,0,0,1,0,59,0


In [ ]:
tds.describe()

,Диагноз,Ёмкость_легких,Объем_выдоха,Шкала_Зуброда,Боль,Кровохарканье,Одышка,Кашель,Слабость,Размер_опухоли,Диабет,ИМ_6_мес,Заболевание_артерий,Курение,Астма,Возраст,Выживание_год
count,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000
mean,3.095745,3.281638,4.568702,0.780851,0.065957,0.144681,0.065957,0.687234,0.165957,1.736170,0.074468,0.004255,0.017021,0.821277,0.004255,62.534043,0.148936
std,0.722309,0.871395,11.767857,0.535375,0.248472,0.352154,0.248472,0.464114,0.372439,0.702243,0.262811,0.065163,0.129488,0.383529,0.065163,8.706902,0.356405
min,1.000000,1.440000,0.960000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.000000,0.000000
25%,3.000000,2.600000,1.960000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,57.000000,0.000000
50%,3.000000,3.160000,2.400000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,62.000000,0.000000
75%,3.000000,3.807500,3.080000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,69.000000,0.000000
max,8.000000,6.300000,86.300000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,87.000000,1.000000


In [ ]:
tds['Диагноз'].value_counts()

3    349
2     52
4     47
5     15
6      4
8      2
1      1
Name: Диагноз, dtype: int64

In [ ]:
test = tds.copy()
test['Диагноз'] = np.where(tds['Диагноз'].isin([1, 8, 6, 5]), 1, tds['Диагноз'])

In [ ]:
test['Диагноз'].value_counts()

3    349
2     52
4     47
1     22
Name: Диагноз, dtype: int64

In [ ]:
X, y = test.drop('Диагноз', axis=1), test.Диагноз

In [ ]:

parameters = {'n_estimators': range(10, 50, 10), 'max_depth': [1, 3, 5],
             'min_samples_leaf': [1, 2, 3], 'min_samples_split': [2, 4],
             'random_state': [0], 'n_jobs': [-1]}
clf = RandomForestClassifier()


In [ ]:
grid = GridSearchCV(clf, parameters, cv=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
grid.fit(X_train, y_train)
predictions = grid.predict(X_test)
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00        16
           3       0.74      1.00      0.85       105
           4       0.00      0.00      0.00        14

    accuracy                           0.74       141
   macro avg       0.19      0.25      0.21       141
weighted avg       0.55      0.74      0.64       141



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
best_grid = grid.best_estimator_
best_grid_features = best_grid.feature_importances_
feature_importances = pd.DataFrame({'features': list(X),
                                   'feature_importances':best_grid_features })
feature_importances.sort_values('feature_importances', ascending=False)

,features,feature_importances
0,Ёмкость_легких,0.249535
1,Объем_выдоха,0.227959
8,Размер_опухоли,0.200427
14,Возраст,0.136993
4,Кровохарканье,0.044651
7,Слабость,0.031933
15,Выживание_год,0.024509
12,Курение,0.024077
5,Одышка,0.020798
3,Боль,0.020610


In [ ]:
test = test.drop(['Астма', 'ИМ_6_мес'], axis=1)

In [ ]:
 X, y = test.drop('Диагноз', axis=1), test.Диагноз

In [ ]:
lr = LogisticRegression(class_weight = 'balanced', max_iter=10000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
lr.fit(X_train, y_train)
predictions = lr.predict(X_test)
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           1       0.08      0.40      0.14         5
           2       0.24      0.50      0.32        10
           3       0.86      0.44      0.58        70
           4       0.08      0.11      0.09         9

    accuracy                           0.41        94
   macro avg       0.31      0.36      0.28        94
weighted avg       0.68      0.41      0.49        94



In [ ]:
svm_lin = SVC(kernel='linear', probability=True)
svm_lin.fit(X_train, y_train)
predictions = svm_lin.predict(X_test)
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00        10
           3       0.74      1.00      0.85        70
           4       0.00      0.00      0.00         9

    accuracy                           0.74        94
   macro avg       0.19      0.25      0.21        94
weighted avg       0.55      0.74      0.64        94



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00        10
           3       0.76      0.97      0.85        70
           4       0.00      0.00      0.00         9

    accuracy                           0.72        94
   macro avg       0.19      0.24      0.21        94
weighted avg       0.56      0.72      0.63        94



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from imblearn.over_sampling import ADASYN

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

augm = ADASYN()
X_train_augm, y_train_augm = augm.fit_resample(np.array(X_train), np.array(y_train))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:

X_test.head()

,Ёмкость_легких,Объем_выдоха,Шкала_Зуброда,Боль,Кровохарканье,Одышка,Кашель,Слабость,Размер_опухоли,Диабет,Заболевание_артерий,Курение,Возраст,Выживание_год
173,2.60,2.36,1,0,0,0,1,0,1,0,0,1,55,1
73,6.30,5.48,0,0,0,0,0,0,1,0,0,0,45,0
111,4.00,2.60,1,0,0,0,1,0,2,1,0,1,58,0
58,5.12,4.28,0,0,0,0,0,0,2,0,0,1,62,0
340,3.88,2.92,0,0,0,0,0,0,1,0,0,1,67,1


In [ ]:
from scipy.stats import uniform
lr1 = LogisticRegression(solver='saga', tol=1e-2, max_iter=10000)



parameters = dict(C=uniform(loc=0, scale=4),
                      penalty=['l2', 'l1'])
clf = RandomizedSearchCV(lr1, parameters, random_state=0, refit=True)
search = clf.fit(X_train_augm, y_train_augm)
search.best_params_
clf.fit(X_train_augm, y_train_augm)
clf = clf.best_estimator_

predictions = clf.predict(X_test)

print(search.best_params_)
print(metrics.classification_report(y_test, predictions))


{'C': 0.22685190926977272, 'penalty': 'l2'}
              precision    recall  f1-score   support

           1       0.06      0.50      0.11         4
           2       0.20      0.45      0.28        11
           3       0.87      0.19      0.31        70
           4       0.00      0.00      0.00         9

    accuracy                           0.21        94
   macro avg       0.28      0.29      0.17        94
weighted avg       0.67      0.21      0.27        94



In [ ]:
svm_lin1 = SVC(kernel='linear', probability=True)

parameters = dict(kernel = ['linear', 'poly', 'rbf', 'sigmoid'], degree=[3, 5])
clf1 = GridSearchCV(svm_lin1, parameters, cv=3)
clf1.fit(X_train_augm, y_train_augm)
best_clf1 = clf1.best_estimator_

print(clf1.best_params_)
predictions = best_clf1.predict(X_test)
print(metrics.classification_report(y_test, predictions))

{'degree': 3, 'kernel': 'linear'}
              precision    recall  f1-score   support

           1       0.16      0.75      0.26         4
           2       0.36      0.36      0.36        11
           3       0.89      0.44      0.59        70
           4       0.17      0.56      0.26         9

    accuracy                           0.46        94
   macro avg       0.39      0.53      0.37        94
weighted avg       0.73      0.46      0.52        94



In [ ]:
knn = KNeighborsClassifier()

parameters = dict(n_neighbors = [x for x in range(1,5)], weights = ['uniform', 'distance'], algorithm = ['ball_tree', 'kd_tree'])
clf = RandomizedSearchCV(knn, parameters, random_state=0, refit=True)
clf.fit(X_train_augm, y_train_augm)
best_clf = clf.best_estimator_

predictions = best_clf.predict(X_test)

print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           1       0.14      0.50      0.22         4
           2       0.14      0.18      0.16        11
           3       0.80      0.67      0.73        70
           4       0.00      0.00      0.00         9

    accuracy                           0.54        94
   macro avg       0.27      0.34      0.28        94
weighted avg       0.62      0.54      0.57        94



In [ ]:
grid.fit(X_train, y_train)
predictions = grid.predict(X_test)
print(grid.best_params_)
print(metrics.classification_report(y_test, predictions))

{'max_depth': 1, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 10, 'n_jobs': -1, 'random_state': 0}
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00        11
           3       0.74      1.00      0.85        70
           4       0.00      0.00      0.00         9

    accuracy                           0.74        94
   macro avg       0.19      0.25      0.21        94
weighted avg       0.55      0.74      0.64        94



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
В итоге можем заключить, что все можели работают довольно плохо, но лучше всего себя показывает метод опорных векторов с линейным ядром, который в некоторых случаях даёт prec = 0.4 и recall = 0.5. 
Немного хуже работает логистическая регрессия.
Данные предсказаны довольно точно для диагноза номер 3, остальные диагнозы, кроме второго, получили случайное предсказание.

{'max_depth': 1,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 10,
 'n_jobs': -1,
 'random_state': 0}